In [1]:
import pandas
pandas.__version__

u'0.21.0'

In [2]:
from collections import defaultdict
inps = {}
outs = {}
txs = {}

def readIntoDict(d,f_name,key_index):
    with open(f_name,'r') as f:
        for l in f:
            arr=l.split(',')
            k = int(arr[key_index])
            del arr[key_index]
            d[k] = tuple(map(int,arr))
readIntoDict(inps,'inputs.csv',0)
readIntoDict(outs,'outputs.csv',0)
readIntoDict(txs,'transactions.csv',0)


In [3]:
inps_by_tx = defaultdict(list)
inps_by_out = defaultdict(list)
outs_by_tx = defaultdict(list)
txs_by_blk = defaultdict(list)

def groupByFirstColumn(dict_new,dict_id,i=0):
    for k,v in dict_id.iteritems():
          dict_new[v[i]].append(k)
groupByFirstColumn(inps_by_tx,inps)
groupByFirstColumn(inps_by_out,inps,i=1)
groupByFirstColumn(outs_by_tx,outs)
groupByFirstColumn(txs_by_blk,txs)


In [4]:
class Checker(object):
    def __init__(self):
        self.txo = set() #outputs that is spent
    
    def checkInputs(s,t_id,block_id):
        tx_inputs = [inps[i][-1] for i in inps_by_tx[t_id]]
        tx_outputs = [o for o in outs_by_tx[t_id]]
        if len(set(tx_inputs) & set(tx_outputs) ) != 0:
            print 'ERROR:Output transaction cannot be input'
            print t_id,tx_inputs,tx_outputs,block_id
            return False
        if len(tx_inputs) != len(set(tx_inputs)): 
            print 'ERROR:There are two inputs pointing the same previous output'
            print t_id,tx_inputs,block_id
            return False
        for o in tx_inputs:
            if o not in outs:
                print 'ERROR:Output script is not correct/doesn\'t exists' 
                print t_id,tx_inputs,block_id
                return False
#         for i_id in tx_inputs:
#             prev_out = outs[inps[i_id][1]]
#             isCoinbase = txs[prev_out[0]][-1] #prev_out[0] -> tx_id [-1]->iscoinbase
#             if isCoinbase and block_id-txs[prev_out[0]][0]<100: #[0]->block_id
#                 print 'You cant spend a reward before 100 confirmation'
#                 print 'Block','Transaction_id','input_id'
#                 print block_id,t_id,i_id
#                 return False   
        return True
    
    def checkBlockSum(s,tx_ids,block_id):
        output_sum = input_sum = 0
        for t_id in tx_ids:
            tx = txs[t_id]
            if not s.checkInputs(t_id,block_id):
                return False
            tx_inputs = set(inps_by_tx[t_id])
            tx_outputs = set(outs_by_tx[t_id])
            c
            for i_id in tx_inputs:
                if inps[i_id][-1] not in s.txo: #in current outputs or previous
                    input_sum += outs[inps[i_id][1]][-1] #[1] for output_id, [-1] for value
                else:
                    print 'ERROR:output %d already used. Called from i_id: %d,t_id: %d' % (inps[i_id][1],i_id,t_id)
                    return False
            for o_id in tx_outputs:
                    output_sum += outs[o_id][-1] #[-1] for value
        if output_sum - input_sum > 5000000000:
            print 'ERROR:Sum is not correct'
            print output_sum - input_sum 
            return False
        else:
            s.txo.update(set([inps[i][-1] for i in tx_inputs]))
            return True
    def checkBlockBasic(s,block_id):
        tx_ids = txs_by_blk[block_id]

        if len(set(tx_ids)) != len(tx_ids):
            print 'ERROR:Error multiple txs with same id!'
            print tx_ids
            return False
        coinbase_txs = [i for i in tx_ids if txs[i][-1]==1]
        if len(coinbase_txs) != 1:
            print 'ERROR:Multiple coinbase transactions!'
            print coinbase_txs
            return False
        coinbase_output_sum = sum([outs[o][-1] for o in (outs_by_tx[coinbase_txs[0]])])
        if coinbase_output_sum < 5000000000:
            print 'WARNING coinbase sum is less than the reward, wth might be wrong', coinbase_txs[0]
        if inps_by_tx[coinbase_txs[0]]:
            print 'ERROR:Coinbase transactions can\'t have input!'
            print 'tx: %d, b_id: %d ' % (coinbase_txs[0],block_id)
            return False
        if not s.checkBlockSum(tx_ids,block_id):
            return False
        return True
                

In [5]:
cc = Checker()
cc.checkBlockBasic(30446)

True

In [6]:
from collections import Counter
no_unique = [(k,v) for k,v in Counter([v[-1] for v in inps.values()]).items() if v!=1]
print no_unique
inps_appended =  [(inps_by_out[i],i) for i,_ in no_unique]
for iss,dup_out in inps_appended:
    for i in iss:
        print i,inps[i]

[(7998, 2), (21928, 2), (65403, 2), (249860, 2)]
521 (8231, 7998)
777 (12152, 7998)
2889 (30446, 21928)
2890 (30446, 21928)
20180 (61843, 65403)
20183 (61845, 65403)
179006 (204751, 249860)
182185 (207365, 249860)


In [7]:
from collections import Counter
cc = Checker()
invalidBlocks = set()
for k in sorted(txs_by_blk.keys()):
    if not cc.checkBlockBasic(k):
        print k,txs_by_blk[k]
        print '---------'
        invalidBlocks.add(k)
#print cc.txo

ERROR:Sum is not correct
5000000010
11181 [11281, 11282]
---------
ERROR:output 7998 already used. Called from i_id: 777,t_id: 12152
12042 [12151, 12152]
---------
ERROR:Output script is not correct/doesn't exists
15698 [265834] 15567
15567 [15697, 15698]
---------
ERROR:There are two inputs pointing the same previous output
30446 [21928, 21928] 30223
30223 [30445, 30446]
---------
ERROR:Multiple coinbase transactions!
[56851, 56853]
52534 [56851, 56852, 56853]
---------
ERROR:Sum is not correct
5000000010
72902 [100928, 100929]
---------
ERROR:Output transaction cannot be input
137237 [156879, 166481] [166481, 166482] 88755
88755 [137236, 137237]
---------
ERROR:output 249860 already used. Called from i_id: 182185,t_id: 207365
97423 [207356, 207357, 207358, 207359, 207360, 207361, 207362, 207363, 207364, 207365, 207366, 207367, 207368]
---------


In [21]:
class GraphSearch(object):
    def __init__(s,invalid_blocks=set()):
        s.invalid_blocks = invalid_blocks
        s.reset_search()
    def reset_search(s):
        s.seen_transactions = set()
        for inv_b in s.invalid_blocks:
            s.seen_transactions.update(txs_by_blk[inv_b])
        s.utxo = set()
    def dfs(s):
        def dfs_helper(tx_id):
            if tx_id not in s.seen_transactions:
                #print tx_id
                is_coinbase = txs[tx_id][1]
                if is_coinbase:
                    s.utxo.update(outs_by_tx[tx_id])
                else:
                    tx_inputs_outs = [inps[i][-1] for i in inps_by_tx[tx_id]]
                    for tx_inp_out in tx_inputs_outs:
                        dfs_helper(outs[tx_inp_out][0]) #tx of the output input points to. 
                        if tx_inp_out not in s.utxo:
                            print 'ERROR input cannot be found, tx_id,tx_inp:%d,%d' %(tx_id,tx_inp_out)
                    s.utxo.update(outs_by_tx[tx_id])
                    s.utxo.difference_update(tx_inputs_outs)
                s.seen_transactions.add(tx_id)
                            
        for tx_id in txs.keys():
            dfs_helper(tx_id)


In [22]:
invalidBlocks

{11181, 12042, 15567, 30223, 52534, 72902, 88755, 97423}

In [23]:
gs = GraphSearch(invalidBlocks)
gs.dfs()

ERROR input cannot be found, tx_id,tx_inp:61845,65403
ERROR input cannot be found, tx_id,tx_inp:114916,60228
ERROR input cannot be found, tx_id,tx_inp:204751,249860
ERROR input cannot be found, tx_id,tx_inp:207370,249869
ERROR input cannot be found, tx_id,tx_inp:208063,249859
ERROR input cannot be found, tx_id,tx_inp:208084,249863
ERROR input cannot be found, tx_id,tx_inp:208477,249852
ERROR input cannot be found, tx_id,tx_inp:208644,249853
ERROR input cannot be found, tx_id,tx_inp:209913,249855
ERROR input cannot be found, tx_id,tx_inp:211792,249857
ERROR input cannot be found, tx_id,tx_inp:213613,249856


In [171]:
print max(((outs[a][-1],a) for a in gs.utxo))

(5000000000000, 236374)


In [10]:
ii = inps_by_out[65403]
tt = [inps[i][0] for i in ii]

In [12]:
print tt
print [txs[t][0] for t in tt]

[61843, 61845]
[56565, 56565]


In [14]:
print txs_by_blk[56565]

[61841, 61842, 61843, 61844, 61845]
